In [1]:
import sys
sys.path.insert(0, "/home/ubuntu/data/code/Modules/")

In [ ]:
import CNN_Module as cnn_module
import DataGenerator_3d
import numpy as np
import random
import models
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

Using TensorFlow backend.


In [ ]:
import imp
imp.reload(DataGenerator_3d)

In [34]:
#prep cells for feeding into CNN
control_cells = np.array(cnn_module.findallcells_indir('/home/ubuntu/data/reistant/'))
sus_cells     = np.array(cnn_module.findallcells_indir('/home/ubuntu/data/susceptible/'))

#label cells, and split cells into test and train, making sure to have equal proportion of sus and ctrl in both
control_cells_label = cnn_module.create_label_dict(control_cells,0)
sus_cells_label  = cnn_module.create_label_dict(sus_cells,1)

train_ctrl, test_ctrl = cnn_module.split_train_test(control_cells_label,0.9)
train_sus, test_sus = cnn_module.split_train_test(sus_cells_label,0.9)

train_labels = train_ctrl+train_sus
test_labels = test_ctrl+test_sus

video_path = '/cropped_video80'
sample_gap = 10
depth = 10
#im_paths_train, im_labels_train = cnn_module.get_labels_images_3d(train_labels,test_labels,video_path,sample_gap,depth)
im_paths_train, im_labels_train, im_paths_test, im_labels_test = cnn_module.get_labels_images_3d(train_labels,test_labels,video_path,sample_gap,depth)
random.shuffle(im_paths_train)
random.shuffle(im_paths_test)


In [42]:
augment_train, augment_valid = cnn_module.get_augmentations_train_test()
params_train, params_test = cnn_module.get_params_train_test(80,augment_train,augment_valid)
params_train['dim']=(10,80,80)
params_test['dim']=(10,80,80)

In [43]:
params_train

{'dim': (10, 80, 80),
 'batch_size': 32,
 'n_classes': 2,
 'augmentations': Compose([
   HorizontalFlip(always_apply=False, p=0.5),
   VerticalFlip(always_apply=False, p=0.5),
   ToFloat(always_apply=False, p=1.0, max_value=65535),
 ], p=1.0, bbox_params=None, keypoint_params=None, additional_targets={}),
 'n_channels': 1,
 'shuffle': True}

In [44]:
training_generator = DataGenerator_3d.DataGenerator(im_paths_train, im_labels_train, **params_train,depth=depth)
prediction_generator = DataGenerator_3d.DataGenerator(im_paths_test, im_labels_test, **params_test,depth=depth)
model = models.get_3d_model((80,80,10,1))

In [45]:
sample_gap=10 #from 10
epochs=40 #from 40
stepsperepoch=100
video_path = '/cropped_video80'
validate_steps = len(im_paths_test)//32
fully_train_model = cnn_module.fit_model(model,training_generator,prediction_generator,epochs,stepsperepoch,validate_steps,params_train,params_test)

Epoch 1/40


ValueError: Error when checking input: expected conv3d_12_input to have shape (80, 80, 10, 1) but got array with shape (10, 80, 80, 1)

In [ ]:
c = cnn_module.predict_3d(model,prediction_generator)
c = np.array(c)

In [ ]:
confusion_matrix(a[:,0],a[:,1])

In [ ]:
b = cnn_module.predict_3d(model,prediction_generator)
b = np.array(b)

In [ ]:
confusion_matrix(b[:,0],b[:,1])

In [ ]:
confusion_matrix(c[:,0],c[:,1])

In [ ]:
(1204+557)/(1204+175+248+557)